# Implementación del algoritmo de detección basado en un autoencoder

Carga de librerías externas, nombre del archivo con espectros de entrenamiento y número de archivos de recolección de datos.

In [ ]:
import math
import json
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from random import sample, randrange
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, f1_score
from keras.layers import Dense, Dropout, Normalization, LSTM, TimeDistributed, RepeatVector

params = [
    {
        'coef_adev': 1,
    },
    {
        'coef_adev': 2,
    },
    {
        'coef_adev': 3,
    },
    {
        'coef_adev': 4,
    },
    {
        'coef_adev': 5,
    },
]

# Posición más alta del vector que actúa como eje X
max_x = 1024

archivo_entrenamiento = '../resultados_entrenamiento.ndjson'
ruta_guardado_modelo = './modelo_entrenado'

numero_archivos_inferencia = 10

config_modelo_propio = {
    'pasos_por_muestra': 256,
    'tasa_dropout': 0.2,
    'epochs': 500,
    'batch_size': 32,
    'proporcion_conjunto_prueba': 0.1,
    'proporcion_validacion': 0.1,
    'min_delta_early_stopping': 0.001,
    'paciencia_early_stopping': 10,
    'cargar_modelo': False
}

Funciones de carga de espectros.

In [ ]:
def deserializar_espectro(espectro):
    espectro['vector_base'] = np.array(espectro['vector_base'])
    espectro['espectro_base_muestra'] = np.array(espectro['espectro_base_muestra'])
    espectro['espectro_base_background'] = np.array(espectro['espectro_base_background'])
    espectro['baseline_muestra'] = np.array(espectro['baseline_muestra'])
    espectro['baseline_background'] = np.array(espectro['baseline_background'])
    espectro['muestra_con_baseline'] = np.array(espectro['muestra_con_baseline'])
    espectro['background_con_baseline'] = np.array(espectro['background_con_baseline'])
    espectro['muestra_combinado_base'] = np.array(espectro['muestra_combinado_base'])
    espectro['espectro_ruido_combinado'] = np.array(espectro['espectro_ruido_combinado'])
    espectro['espectro_ruido_background'] = np.array(espectro['espectro_ruido_background'])
    espectro['spikes_muestra'] = np.array(espectro['spikes_muestra'])
    espectro['spikes_background'] = np.array(espectro['spikes_background'])
    espectro['flag_spikes_muestra'] = np.array(espectro['flag_spikes_muestra'])
    espectro['flag_spikes_background'] = np.array(espectro['flag_spikes_background'])
    espectro['muestra_base_con_spikes'] = np.array(espectro['muestra_base_con_spikes'])
    espectro['y_muestra'] = np.array(espectro['y_muestra'])
    espectro['y_background'] = np.array(espectro['y_background'])

    return espectro

def cargar_espectros(nombre_archivo):
    with open(nombre_archivo, 'r') as fp:
        data = []
        for line in fp:
            data.append(deserializar_espectro(json.loads(line)))
        fp.close()
        return data

espectros = cargar_espectros(archivo_entrenamiento)

Función para calcular métricas de clasificación de spikes.

In [ ]:
def calcular_metricas_clasificacion(valores_reales, predicciones):
    _predicciones = np.array(predicciones).flatten()
    _valores_reales = np.array(valores_reales).flatten()

    return precision_recall_fscore_support(_valores_reales, _predicciones), confusion_matrix(_valores_reales, _predicciones)

Separación del conjunto de datos en conjuntos de prueba y entrenamiento

In [ ]:
espectros_entrenamiento, espectros_prueba = train_test_split(espectros, test_size=config_modelo_propio['proporcion_conjunto_prueba'])

etiquetas_entrenamiento = [espectro['spikes_muestra'] for espectro in espectros_entrenamiento]
etiquetas_prueba = [espectro['spikes_muestra'] for espectro in espectros_prueba]

muestras_ruido_multiple_entrenamiento = [espectro['muestra_base_con_spikes'] for espectro in espectros_entrenamiento]
muestras_ruido_multiple_prueba = [espectro['muestra_base_con_spikes'] for espectro in espectros_prueba]

Implementación de un algoritmo basado en un denoising autoencoder para detección de spikes, así como funciones auxiliares para recogida de datos y visualización de resultados.

In [ ]:
def transformar_compatible_lstm(x, y, pasos_por_muestra):
    x_compatible = np.array(x)
    x_compatible = x_compatible.reshape(
        (x_compatible.shape[0], pasos_por_muestra, round(x_compatible.shape[1] / pasos_por_muestra))
    )
    
    y_compatible = np.array(y)
    y_compatible = y_compatible.reshape(
        (y_compatible.shape[0], pasos_por_muestra, round(y_compatible.shape[1] / pasos_por_muestra))
    )
    
    return x_compatible, y_compatible

def crear_modelo_propio_entrenado(X_entrenamiento, config_modelo):
    ventana = config_modelo['pasos_por_muestra']
    tasa_dropout = config_modelo['tasa_dropout']
    epochs = config_modelo['epochs']
    batch_size = config_modelo['batch_size']
    min_delta = config_modelo['min_delta_early_stopping']
    paciencia = config_modelo['paciencia_early_stopping']
    proporcion_validacion = config_modelo['proporcion_validacion']
    
    X, _ = transformar_compatible_lstm(
        X_entrenamiento,
        X_entrenamiento,
        ventana
    )

    shape = X.shape
    modelo = Sequential()
    normalizador = Normalization(input_shape=(shape[1], shape[2]))

    normalizador.adapt(X)

    modelo.add(normalizador)

    modelo.add(LSTM(ventana))
    modelo.add(Dropout(rate=tasa_dropout))

    modelo.add(RepeatVector(shape[1]))

    modelo.add(LSTM(ventana, return_sequences=True))
    modelo.add(TimeDistributed(Dense(shape[2])))
    modelo.compile(optimizer='adam', loss='mean_squared_error')
    modelo.summary()

    callback = EarlyStopping(
        monitor="val_loss",
        min_delta=min_delta,
        patience=paciencia,
        verbose=0,
        mode="auto",
        baseline=None,
        restore_best_weights=True,
    )
    
    historia = modelo.fit(
        X,
        X,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=proporcion_validacion,
        verbose=1,
        callbacks=[callback]
    )
    
    return modelo, historia

def realizar_inferencia_modelo_propio(modelo_entrenado, X, config_modelo, coef_adev = 3):
    ventana = config_modelo['pasos_por_muestra']
    X, _ = transformar_compatible_lstm(
        X,
        X,
        ventana
    )
    
    prediccion_cruda = modelo_entrenado.predict(X)
    
    detecciones = []
    predicciones = []
    residuales = []
    umbrales_adev = []
        
    for i in range(len(X)):
        datos_originales = X[i].flatten()
        prediccion = prediccion_cruda[i].flatten()
        
        serie = datos_originales - prediccion
        
        serie_diferenciada = np.diff(serie)
        
        segunda_diferencia = np.diff(serie_diferenciada)
        suma_segundas_diferencias_cuadrado = np.sum(segunda_diferencia ** 2)
        adev = math.sqrt(suma_segundas_diferencias_cuadrado / (2 * (len(segunda_diferencia) - 2)))
                
        umbral_adev = coef_adev * adev

        deteccion = np.abs(serie_diferenciada) > umbral_adev
        
        deteccion = np.insert(deteccion, 0, False)
                
        detecciones.append(deteccion)
        predicciones.append(prediccion)
        residuales.append(serie_diferenciada)
        umbrales_adev.append(umbral_adev)
            
    return detecciones, predicciones, residuales, umbrales_adev

def obtener_resultado_modelo_propio(modelo_entrenado, X, y, config_modelo, coef_adev = 3):
    predicciones, _, _, _ = realizar_inferencia_modelo_propio(
        modelo_entrenado,
        X,
        config_modelo,
        coef_adev
    )
    
    return predicciones, calcular_metricas_clasificacion(y, predicciones)

def visualizar_resultado_modelo_propio(vector_base, modelo_entrenado, X, etiquetas, config_modelo, coef_adev = 3):
    predicciones, forma, residuales, umbrales_adev = realizar_inferencia_modelo_propio(
        modelo_entrenado,
        X,
        config_modelo,
        coef_adev
    )
    
    predicciones = predicciones[0]
    forma = forma[0]
    residuales = residuales[0]
    umbral_adev = umbrales_adev[0]
    
    fig = plt.figure(figsize=[45, 15], constrained_layout=True)

    fig.suptitle("Muestra de resultados al aplicar el algoritmo propio", fontsize=24, fontweight='bold')

    subfigs = fig.subfigures(nrows=3, ncols=1)

    for row, subfig in enumerate(subfigs):
        (ax) = subfig.subplots(nrows=1, ncols=1)

        if row == 0:
            ax.set_title("Serie original", fontsize=18)
            ax.plot(vector_base, X[0], label="Serie original")
            ax.plot(vector_base, forma, label="Serie reconstruida por el modelo", c='r')
            ax.legend(fontsize=28)
        elif row == 1:
            ax.set_title("Serie de valores residuales y umbral", fontsize=18)
            residuales = np.insert(residuales, 0, 0)
            ax.plot(vector_base, residuales)
            plt.axhline(y = umbral_adev, color = 'r', linestyle = '-')
            plt.axhline(y = -umbral_adev, color = 'r', linestyle = '-')
        elif row == 2:
            categorias = {
                'tp': {
                    'color': 'blue',
                    'label': 'Detecciones'
                },
                'fp': {
                    'color': 'black',
                    'label': 'Falsos positivos'
                },
                'fn': {
                    'color': 'red',
                    'label': 'Falsos negativos'
                }
            }

            detecciones_agrupadas = []

            for i, es_resultado in enumerate(predicciones):
                label = ''
                es_spike = etiquetas[i]

                if es_spike:
                    if es_resultado:
                        label = 'tp'
                    else:
                        label = 'fn'
                elif es_resultado:
                    label = 'fp'
                else:
                    label = 'tn'
                detecciones_agrupadas.append(label)

            detecciones_agrupadas = np.array(detecciones_agrupadas)

            ax.set_title("Serie con spikes detectados y tipos de detección", fontsize=18)
            ax.plot(vector_base, X[0])
            ax.scatter(vector_base[detecciones_agrupadas == 'tp'], X[0][detecciones_agrupadas == 'tp'], c=categorias['tp']['color'], label=categorias['tp']['label'], s=100)
            ax.scatter(vector_base[detecciones_agrupadas == 'fp'], X[0][detecciones_agrupadas == 'fp'], c=categorias['fp']['color'], label=categorias['fp']['label'], s=100)
            ax.scatter(vector_base[detecciones_agrupadas == 'fn'], X[0][detecciones_agrupadas == 'fn'], c=categorias['fn']['color'], label=categorias['fn']['label'], s=100)

            ax.legend(fontsize=28)

In [ ]:
modelo_propio_entrenado = None
historial_entrenamiento = None

if config_modelo_propio['cargar_modelo']:
    modelo_propio_entrenado = tf.keras.models.load_model(ruta_guardado_modelo)
else:
    modelo_propio_entrenado, historial_entrenamiento = crear_modelo_propio_entrenado(
        muestras_ruido_multiple_entrenamiento,
        config_modelo_propio
    )

    modelo_propio_entrenado.save(ruta_guardado_modelo)

Prueba del modelo entrenado, para detectar posible overfitting

In [ ]:
X_prueba, y_prueba = transformar_compatible_lstm(
    muestras_ruido_multiple_prueba,
    etiquetas_prueba,
    config_modelo_propio['pasos_por_muestra'],
)

if not config_modelo_propio['cargar_modelo']:
    resultado_evaluacion_modelo = modelo_propio_entrenado.evaluate(X_prueba, y_prueba, verbose=2)
    with open('historial_entrenamiento_modelo_propio', 'wb') as fp:
        pickle.dump({
            'historial_entrenamiento': historial_entrenamiento,
            'validacion': resultado_evaluacion_modelo
        }, fp)
        fp.close()

Recogida de datos y guardado de los resultados en un archivo.

In [ ]:
resultados_modelo_propio_ruido = []
resultados_modelo_propio_sin_ruido = []

def procesar_archivo(n):
    nombre_archivo = '../resultados_' + str(n) + '.ndjson'

    resultados_ruido = []
    resultados_sin_ruido = []

    espectros = cargar_espectros(nombre_archivo)

    espectros_base_con_spikes_con_ruido = []
    espectros_base_con_spikes_sin_ruido = []
    etiquetas = []

    for espectro in espectros:
        espectros_base_con_spikes_con_ruido.append(espectro['muestra_base_con_spikes'])
        espectros_base_con_spikes_sin_ruido.append(espectro['espectro_base_muestra'] + espectro['spikes_muestra'])
        etiquetas.append(espectro['flag_spikes_muestra'])

    predicciones_modelo_propio_ruido, metricas_modelo_propio_ruido = obtener_resultado_modelo_propio(
        modelo_propio_entrenado,
        espectros_base_con_spikes_con_ruido,
        etiquetas,
        config_modelo_propio,
        coef_adev=param['coef_adev'],
    )
    predicciones_modelo_propio_sin_ruido, metricas_modelo_propio_sin_ruido = obtener_resultado_modelo_propio(
        modelo_propio_entrenado,
        espectros_base_con_spikes_sin_ruido,
        etiquetas,
        config_modelo_propio,
        coef_adev=param['coef_adev'],
    )

    resultados_ruido.append({
        'precision_negativos': metricas_modelo_propio_ruido[0][0][0],
        'precision_positivos': metricas_modelo_propio_ruido[0][0][1],
        'support_negativos': metricas_modelo_propio_ruido[0][1][0],
        'support_positivos': metricas_modelo_propio_ruido[0][1][1],
        'f1_negativos': metricas_modelo_propio_ruido[0][2][0],
        'f1_positivos': metricas_modelo_propio_ruido[0][2][1],
        'vn': metricas_modelo_propio_ruido[1][0][0],
        'fp': metricas_modelo_propio_ruido[1][0][1],
        'fn': metricas_modelo_propio_ruido[1][1][0],
        'vp': metricas_modelo_propio_ruido[1][1][1]
    })

    resultados_sin_ruido.append({
        'precision_negativos': metricas_modelo_propio_sin_ruido[0][0][0],
        'precision_positivos': metricas_modelo_propio_sin_ruido[0][0][1],
        'support_negativos': metricas_modelo_propio_sin_ruido[0][1][0],
        'support_positivos': metricas_modelo_propio_sin_ruido[0][1][1],
        'f1_negativos': metricas_modelo_propio_sin_ruido[0][2][0],
        'f1_positivos': metricas_modelo_propio_sin_ruido[0][2][1],
        'vn': metricas_modelo_propio_sin_ruido[1][0][0],
        'fp': metricas_modelo_propio_sin_ruido[1][0][1],
        'fn': metricas_modelo_propio_sin_ruido[1][1][0],
        'vp': metricas_modelo_propio_sin_ruido[1][1][1]
    })

    return resultados_ruido, resultados_sin_ruido

for param in params:
    resultados_archivo = [procesar_archivo(n) for n in range(numero_archivos_inferencia)]

    resultados_ruido = [resultado[0][0] for resultado in resultados_archivo]
    resultados_sin_ruido = [resultado[1][0] for resultado in resultados_archivo]

    resultados_modelo_propio_ruido.append({
        'param': param,
        'resultados': resultados_ruido
    })

    resultados_modelo_propio_sin_ruido.append({
        'param': param,
        'resultados': resultados_sin_ruido
    })

with open('resultados_modelo_propio_ruido', 'wb') as fp:
    pickle.dump(resultados_modelo_propio_ruido, fp)
    fp.close()

with open('resultados_modelo_propio_sin_ruido', 'wb') as fp:
    pickle.dump(resultados_modelo_propio_sin_ruido, fp)
    fp.close()

Aquí obtenemos y guardamos en un archivo los peores resultados obtenidos por el algoritmo con sus mejores parámetros (con o sin ruido) para posterior análisis

In [ ]:
lista_medias_f1_positivos_ruido = [np.mean([resultado['f1_positivos'] for resultado in resultado_param['resultados']]) for resultado_param in resultados_modelo_propio_ruido]
indice_mejores_params_ruido = np.argmax(lista_medias_f1_positivos_ruido)
mejores_params_ruido = params[indice_mejores_params_ruido]

lista_medias_f1_positivos_sin_ruido = [np.mean([resultado['f1_positivos'] for resultado in resultado_param['resultados']]) for resultado_param in resultados_modelo_propio_sin_ruido]
indice_mejores_params_sin_ruido = np.argmax(lista_medias_f1_positivos_sin_ruido)
mejores_params_sin_ruido = params[indice_mejores_params_sin_ruido]

archivo_azar = randrange(numero_archivos_inferencia)

nombre_archivo = '../resultados_' + str(archivo_azar) + '.ndjson'

espectros = cargar_espectros(nombre_archivo)

prediccion_con_ruido = realizar_inferencia_modelo_propio(
    modelo_propio_entrenado,
    [espectro['muestra_base_con_spikes'] for espectro in espectros],
    config_modelo_propio,
    coef_adev=mejores_params_ruido['coef_adev']
)[0]

prediccion_sin_ruido = realizar_inferencia_modelo_propio(
    modelo_propio_entrenado,
    [espectro['espectro_base_muestra'] + espectro['spikes_muestra'] for espectro in espectros],
    config_modelo_propio,
    coef_adev=mejores_params_sin_ruido['coef_adev']
)[0]

resultados_f1_ruido = [(
    espectro['muestra_base_con_spikes'],
    f1_score(
        espectro['flag_spikes_muestra'],
        prediccion_con_ruido[i],
        zero_division=1
    ),
    prediccion_con_ruido[i],
    espectro['flag_spikes_muestra'],
    mejores_params_ruido
) for i, espectro in enumerate(espectros)]

resultados_f1_sin_ruido = [(
    espectro['espectro_base_muestra'] + espectro['spikes_muestra'],
    f1_score(
        espectro['flag_spikes_muestra'],
        prediccion_sin_ruido[i],
        zero_division=1
    ),
    prediccion_sin_ruido[i],
    espectro['flag_spikes_muestra'],
    mejores_params_sin_ruido
) for i, espectro in enumerate(espectros)]

resultados_f1_ruido = list(filter(lambda d: d[1] > 0, resultados_f1_ruido))
resultados_f1_sin_ruido = list(filter(lambda d: d[1] > 0, resultados_f1_sin_ruido))

resultados_f1_ruido = sorted(resultados_f1_ruido, key=lambda d: d[1])
resultados_f1_sin_ruido = sorted(resultados_f1_sin_ruido, key=lambda d: d[1])

with open('peores_resultados_modelo_propio', 'wb') as fp:
    peores_resultados_ruido = resultados_f1_ruido[0:10]
    peores_resultados_sin_ruido = resultados_f1_sin_ruido[0:10]
    peores_resultados = {
        'ruido': peores_resultados_ruido,
        'sin_ruido': peores_resultados_sin_ruido
    }
    pickle.dump(peores_resultados, fp)
    fp.close()

Toma de una muestra aleatoria de los resultados para control de calidad

In [ ]:
indice_muestra = sample(list(range(len(espectros))), 1)[0]

Visualización del resultado de la aplicación del algoritmo a un espectro aleatorio al que no le fue añadido ruido aleatorio

In [ ]:
visualizar_resultado_modelo_propio(
    espectros[indice_muestra]['vector_base'],
    modelo_propio_entrenado,
    [espectros[indice_muestra]['espectro_base_muestra'] + espectros[indice_muestra]['spikes_muestra']],
    espectros[indice_muestra]['flag_spikes_muestra'],
    config_modelo_propio,
    coef_adev=mejores_params_sin_ruido['coef_adev']
)

Visualización del resultado de la aplicación del algoritmo a un espectro aleatorio al que le fue añadido ruido aleatorio

In [ ]:
visualizar_resultado_modelo_propio(
    espectros[indice_muestra]['vector_base'],
    modelo_propio_entrenado,
    [espectros[indice_muestra]['muestra_base_con_spikes']],
    espectros[indice_muestra]['flag_spikes_muestra'],
    config_modelo_propio,
    coef_adev=mejores_params_sin_ruido['coef_adev']
)